# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 6 2023 11:20AM,256705,16,SSF_C2W6,Sartorius Stedim Filters Inc.,Released
1,Feb 6 2023 11:11AM,256704,10,9123603,"Snap Medical Industries, LLC",Released
2,Feb 6 2023 11:11AM,256704,10,9123605,"Snap Medical Industries, LLC",Released
3,Feb 6 2023 11:11AM,256704,10,9123606,"Snap Medical Industries, LLC",Released
4,Feb 6 2023 11:11AM,256704,10,9123607,"Snap Medical Industries, LLC",Released
5,Feb 6 2023 11:11AM,256704,10,9123608,"Snap Medical Industries, LLC",Released
6,Feb 6 2023 11:11AM,256704,10,9123609,"Snap Medical Industries, LLC",Released
7,Feb 6 2023 11:11AM,256704,10,9123610,"Snap Medical Industries, LLC",Released
8,Feb 6 2023 11:03AM,256703,12,7559011,"NBTY Global, Inc.",Released
9,Feb 6 2023 11:01AM,256697,20,9122500,"Exact-Rx, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
22,256697,Released,1
23,256698,Released,1
24,256703,Released,1
25,256704,Released,7
26,256705,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
256697,NaN,1.0
256698,NaN,1.0
256703,NaN,1.0
256704,NaN,7.0
256705,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
256656,2.0,1.0
256662,1.0,0.0
256671,0.0,48.0
256673,0.0,36.0
256676,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
256656,2,1
256662,1,0
256671,0,48
256673,0,36
256676,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,256656,2,1
1,256662,1,0
2,256671,0,48
3,256673,0,36
4,256676,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,256656,2,1
1,256662,1,
2,256671,,48
3,256673,,36
4,256676,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 6 2023 11:20AM,256705,16,Sartorius Stedim Filters Inc.
1,Feb 6 2023 11:11AM,256704,10,"Snap Medical Industries, LLC"
8,Feb 6 2023 11:03AM,256703,12,"NBTY Global, Inc."
9,Feb 6 2023 11:01AM,256697,20,"Exact-Rx, Inc."
10,Feb 6 2023 10:54AM,256698,19,"Methapharm, Inc."
11,Feb 6 2023 10:50AM,256696,12,"NBTY Global, Inc."
12,Feb 6 2023 10:40AM,256695,15,"Brookfield Pharmaceuticals, LLC"
46,Feb 6 2023 10:28AM,256694,16,Sartorius Bioprocess Solutions
47,Feb 6 2023 10:22AM,256686,16,Sartorius Stedim Filters Inc.
48,Feb 6 2023 10:15AM,256692,10,Methapharm-G


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Feb 6 2023 11:20AM,256705,16,Sartorius Stedim Filters Inc.,,1
1,Feb 6 2023 11:11AM,256704,10,"Snap Medical Industries, LLC",,7
2,Feb 6 2023 11:03AM,256703,12,"NBTY Global, Inc.",,1
3,Feb 6 2023 11:01AM,256697,20,"Exact-Rx, Inc.",,1
4,Feb 6 2023 10:54AM,256698,19,"Methapharm, Inc.",,1
5,Feb 6 2023 10:50AM,256696,12,"NBTY Global, Inc.",,1
6,Feb 6 2023 10:40AM,256695,15,"Brookfield Pharmaceuticals, LLC",27,7
7,Feb 6 2023 10:28AM,256694,16,Sartorius Bioprocess Solutions,,1
8,Feb 6 2023 10:22AM,256686,16,Sartorius Stedim Filters Inc.,,1
9,Feb 6 2023 10:15AM,256692,10,Methapharm-G,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 6 2023 11:20AM,256705,16,Sartorius Stedim Filters Inc.,1,
1,Feb 6 2023 11:11AM,256704,10,"Snap Medical Industries, LLC",7,
2,Feb 6 2023 11:03AM,256703,12,"NBTY Global, Inc.",1,
3,Feb 6 2023 11:01AM,256697,20,"Exact-Rx, Inc.",1,
4,Feb 6 2023 10:54AM,256698,19,"Methapharm, Inc.",1,
5,Feb 6 2023 10:50AM,256696,12,"NBTY Global, Inc.",1,
6,Feb 6 2023 10:40AM,256695,15,"Brookfield Pharmaceuticals, LLC",7,27
7,Feb 6 2023 10:28AM,256694,16,Sartorius Bioprocess Solutions,1,
8,Feb 6 2023 10:22AM,256686,16,Sartorius Stedim Filters Inc.,1,
9,Feb 6 2023 10:15AM,256692,10,Methapharm-G,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 6 2023 11:20AM,256705,16,Sartorius Stedim Filters Inc.,1,
1,Feb 6 2023 11:11AM,256704,10,"Snap Medical Industries, LLC",7,
2,Feb 6 2023 11:03AM,256703,12,"NBTY Global, Inc.",1,
3,Feb 6 2023 11:01AM,256697,20,"Exact-Rx, Inc.",1,
4,Feb 6 2023 10:54AM,256698,19,"Methapharm, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Feb 6 2023 11:20AM,256705,16,Sartorius Stedim Filters Inc.,1.0,NaN
1,Feb 6 2023 11:11AM,256704,10,"Snap Medical Industries, LLC",7.0,NaN
2,Feb 6 2023 11:03AM,256703,12,"NBTY Global, Inc.",1.0,NaN
3,Feb 6 2023 11:01AM,256697,20,"Exact-Rx, Inc.",1.0,NaN
4,Feb 6 2023 10:54AM,256698,19,"Methapharm, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 6 2023 11:20AM,256705,16,Sartorius Stedim Filters Inc.,1.0,0.0
1,Feb 6 2023 11:11AM,256704,10,"Snap Medical Industries, LLC",7.0,0.0
2,Feb 6 2023 11:03AM,256703,12,"NBTY Global, Inc.",1.0,0.0
3,Feb 6 2023 11:01AM,256697,20,"Exact-Rx, Inc.",1.0,0.0
4,Feb 6 2023 10:54AM,256698,19,"Methapharm, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2053491,108.0,5.0
12,513399,2.0,0.0
15,256695,7.0,27.0
16,1283428,4.0,5.0
19,770030,4.0,2.0
20,256697,1.0,0.0
21,770052,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2053491,108.0,5.0
1,12,513399,2.0,0.0
2,15,256695,7.0,27.0
3,16,1283428,4.0,5.0
4,19,770030,4.0,2.0
5,20,256697,1.0,0.0
6,21,770052,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,108.0,5.0
1,12,2.0,0.0
2,15,7.0,27.0
3,16,4.0,5.0
4,19,4.0,2.0
5,20,1.0,0.0
6,21,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,108.0
1,12,Released,2.0
2,15,Released,7.0
3,16,Released,4.0
4,19,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Executing,5.0,0.0,27.0,5.0,2.0,0.0,0.0
Released,108.0,2.0,7.0,4.0,4.0,1.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Executing,5.0,0.0,27.0,5.0,2.0,0.0,0.0
1,Released,108.0,2.0,7.0,4.0,4.0,1.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Executing,5.0,0.0,27.0,5.0,2.0,0.0,0.0
1,Released,108.0,2.0,7.0,4.0,4.0,1.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()